# Tremor analysis

In [1]:
import pandas as pd
from paradigma.config import IMUConfig, TremorFeatureExtractionConfig, TremorDetectionConfig, TremorQuantificationConfig
from paradigma.imu_preprocessing import preprocess_imu_data
from paradigma.tremor.tremor_analysis import extract_tremor_features, detect_tremor, quantify_tremor

In [2]:
df_tremor_week = pd.DataFrame(columns=['low_freq_power','tremor_power','pred_tremor_checked']) # create dataframe for concatenation of segments after the tremor detection step

## Load data

In [2]:
import tsdf
from paradigma.util import read_metadata

input_path =  'C:\\Users\\z835211\\Documents\\Data\\Raw IMU data\\WatchData.IMU.Week2\\POMU0000775B2E171842'
segment = 'WatchData.IMU.Week2.raw_segment0010'

# Load data
metadata_time, metadata_values = read_metadata(str(input_path), str(segment+'_meta.json'),
                                                    str(segment+'_time.bin'), str(segment+'_samples.bin'))
df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_values], tsdf.constants.ConcatenationType.columns)

# Rename columns
df = df.rename(columns={f'rotation_{a}': f'gyroscope_{a}' for a in ['x', 'y', 'z']})
df = df.rename(columns={f'acceleration_{a}': f'accelerometer_{a}' for a in ['x', 'y', 'z']})

config = IMUConfig()

# Apply scale factors 
df[list(config.d_channels_imu.keys())] *= metadata_values.scale_factors


## Preprocess gyroscope data

In [3]:
config = IMUConfig()
df_preprocessed = preprocess_imu_data(df, config, sensor='gyroscope')

## Extract features

In [ ]:
config = TremorFeatureExtractionConfig()
df_features = extract_tremor_features(df_preprocessed, config)
print(df_features)

## Detect tremor

In [ ]:
config = TremorDetectionConfig()
path_to_classifier_input =  '../../../tests/data/0.classification/tremor'
df_tremor_predictions = detect_tremor(df_features, config, path_to_classifier_input)

df_tremor_week = pd.concat([df_tremor_week, df_tremor_predictions[['low_freq_power','tremor_power','pred_tremor_checked']]],ignore_index=True)


## Quantify tremor

In [ ]:
config = TremorQuantificationConfig()
d_aggregates = quantify_tremor(df_tremor_week, config)

import json
import os 

# Save output
with open(os.path.join(input_path,"weekly_tremor.json"), 'w') as json_file:
    json.dump(d_aggregates, json_file, indent=4)